# import libraries

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import time
import glob
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.framework import convert_to_constants
from tensorflow.keras.preprocessing import image

# load dataset

In [2]:
"""
class_name 은 폴더 이름과 동일하게 설정

2contexts 모델은 road 데이터를 제외하고 훈련된 모델이기 때문에, 
2contexts 모델을 테스트하기 위해서는 class_names 를 다시 설정해주어야 합니다.
"""

class_names = ['construction_danger', 'construction_safe', 'park_danger', 'park_safe', 'road_danger', 'road_safe']
label_list = []
image_list = []
dataset_list = []

for label in class_names:
    img_path_list = glob.glob(f'../dataset/{label}/*')
    # print(img_path_list)
    
    for i in range(len(img_path_list)):
        image = tf.io.read_file(img_path_list[i])
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [224, 224])
        dataset_list.append(tf.expand_dims(image, 0))
        image_list.append(image)
        label_list.append(label)
        
# 이미지가 잘 불러들어와 졌는지 확인        
len(image_list)

600

# set class

In [3]:
class Saved_model_test:
    def __init__(self, saved_model_dir, model_type, class_names):
        self.saved_model_dir = saved_model_dir
        
        start_time = time.time()
        self.saved_model_loaded = tf.keras.models.load_model(saved_model_dir)
        end_time = time.time()
        term = end_time - start_time
        self.loading_term = term
        
        self.class_names = class_names
        self.model_type = model_type
        
        
        '''
        keras 에서 바로 저장한 saved model 과 tf-trt 로 변환 후 저장한 saved model 의 타입이 다르기 때문에
        if-else 로 predict 함수를 설정해줍니다.
        '''
        if self.model_type == 'keras':
            self.infer = self.saved_model_loaded.predict
            self.output_layer = None
        elif self.model_type == 'saved_model':
            self.infer = self.saved_model_loaded.signatures['serving_default']
            self.output_layer = next(iter(self.infer.structured_outputs.keys()))
        
        self.saved_model_infer = {}
        self.infer_result = []
        
    
    def check_speed(self, input):
        N_warmup_run = 50
        N_run = 20
        elapsed_time = []
        
        '''
        가끔, 처음 실행하는 하나의 텀은 속도가 느리게 나오는 경우가 있기 때문에
        처음 N_warmup_up을 실행해줍니다. (속도 측정에 포함시키지 않음) 
        '''
        for i in range(N_warmup_run):
            preds = self.infer(input)
        
        for i in range(N_run):
            start_time = time.time()
            for j in range(N_warmup_run):
                preds = self.infer(input)
            end_time = time.time()
            elapsed_time = np.append(elapsed_time, end_time - start_time)
        self.elapsed_time = elapsed_time
        
        total = 0
        for i in range(N_run):
            total += elapsed_time[i]
        # total /= len(elapsed_time)
        self.speed = (N_run * N_warmup_run)/total
        
    
    def test(self, dataset_list, label_list):
        
        result_list = []
        for i in range(len(dataset_list)):
            if self.model_type == 'keras':
                preds = self.infer(dataset_list[i])    
            elif self.model_type == 'saved_model':
                preds =  self.infer(dataset_list[i])[self.output_layer].numpy()
            result_list.append([label_list[i], self.class_names[np.argmax(preds[0], axis=0)],  preds])
            
        self.result_list = result_list
        
    def calc_accuracy(self):
        self.accuracy = {}
        
        c = np.array(self.result_list)
        for label in self.class_names:
            first_idx = np.where(c[0:, 0:1] == label)[0][0]
            last_idx = np.where(c[0:, 0:1] == label)[0][-1]

            label_len = len(np.where(c[first_idx:last_idx+1, 0:1] == label)[0])
            correct = len(np.where(c[first_idx:last_idx+1:, 1:2] == label)[0])
            self.accuracy[label] = correct/label_len

# test

In [4]:
def test(input, type, class_names):
    model_test = Saved_model_test(input, type, class_names)
    print(f'loading term (sec): {model_test.loading_term}')
    
    model_test.check_speed(dataset_list[0])
    print(f'images per sec: {model_test.speed}')

    model_test.test(dataset_list, label_list)
    model_test.calc_accuracy()
    print(f'accuracy: {model_test.accuracy}')

## custom_saved_model

##### custom_saved_model

In [5]:
test('../saved_models/custom_saved_model', 'keras', class_names)

loading term (sec): 14.144092798233032
images per sec: 9.17250037542418
accuracy: {'construction_danger': 0.55, 'construction_safe': 0.53, 'park_danger': 0.41, 'park_safe': 0.54, 'road_danger': 0.77, 'road_safe': 0.56}


/home/keti/tf_2.5.0/lib/python3.6/site-packages/ipykernel_launcher.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


##### origin - custom_saved_model_TFTRT_F32

In [6]:
test('custom_saved_model_TFTRT_F32', 'saved_model', class_names)

loading term (sec): 57.75183916091919
images per sec: 665.5865290226426
accuracy: {'construction_danger': 0.55, 'construction_safe': 0.53, 'park_danger': 0.41, 'park_safe': 0.54, 'road_danger': 0.77, 'road_safe': 0.56}


/home/keti/tf_2.5.0/lib/python3.6/site-packages/ipykernel_launcher.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


##### origin - custom_saved_model_TFTRT_INT8

In [7]:
test('custom_saved_model_TFTRT_INT8', 'saved_model', class_names)

loading term (sec): 54.12956261634827
images per sec: 588.1278851267854
accuracy: {'construction_danger': 0.55, 'construction_safe': 0.53, 'park_danger': 0.41, 'park_safe': 0.54, 'road_danger': 0.77, 'road_safe': 0.56}


/home/keti/tf_2.5.0/lib/python3.6/site-packages/ipykernel_launcher.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


## custom_resnet_saved_model

##### custom_resnet_saved_model

In [15]:
test('../saved_models/custom_resnet_saved_model', 'keras', class_names)

loading term (sec): 77.23092865943909
images per sec: 7.706925223290779
accuracy: {'construction_danger': 0.54, 'construction_safe': 0.44, 'park_danger': 0.78, 'park_safe': 0.0, 'road_danger': 0.94, 'road_safe': 0.75}


/home/keti/tf_2.5.0/lib/python3.6/site-packages/ipykernel_launcher.py:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


##### custom_resnet_saved_model_TFTRT_F32

In [10]:
test('custom_resnet_saved_model_TFTRT_F32', 'saved_model', class_names)

loading term (sec): 570.6001949310303
images per sec: 460.99332795143766
accuracy: {'construction_danger': 0.55, 'construction_safe': 0.53, 'park_danger': 0.41, 'park_safe': 0.54, 'road_danger': 0.77, 'road_safe': 0.56}


/home/keti/tf_2.5.0/lib/python3.6/site-packages/ipykernel_launcher.py:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


##### custom_resnet_saved_model_TFTRT_INT8

In [12]:
test('custom_resnet_saved_model_TFTRT_INT8', 'saved_model', class_names)

loading term (sec): 635.6248414516449
images per sec: 319.9566220918688


/home/keti/tf_2.5.0/lib/python3.6/site-packages/ipykernel_launcher.py:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


accuracy: {'construction_danger': 0.54, 'construction_safe': 0.45, 'park_danger': 0.79, 'park_safe': 0.1, 'road_danger': 0.94, 'road_safe': 0.74}


## custom_mobilenetv2_saved_model

##### custom_mobilenetv2_saved_model

In [ ]:
test('../saved_models/custom_mobilenetv2_saved_model', 'keras', class_names)

loading term (sec): 219.51848673820496


##### custom_mobilenetv2_saved_model_TFTRT_F32

In [ ]:
test('custom_mobilenetv2_saved_model_TFTRT_F32', 'saved_model', class_names)

##### custom_mobilenetv2_saved_model_TFTRT_INT8

In [ ]:
test('custom_mobilenetv2_saved_model_TFTRT_INT8', 'saved_model', class_names)

## custom_efficientnetb1_saved_model

##### custom_efficientnetb1_saved_model

In [6]:
test('../saved_models/custom_efficientnetb1_saved_model', 'keras', class_names)

loading term (sec): 167.44114708900452
images per sec: 7.173884105213199
accuracy: {'construction_danger': 0.9, 'construction_safe': 1.0, 'park_danger': 1.0, 'park_safe': 0.9, 'road_danger': 0.8, 'road_safe': 0.6}


##### custom_efficientnetb1_saved_model_TFTRT_F32

In [7]:
test('custom_efficientnetb1_saved_model_TFTRT_F32', 'saved_model', class_names)

loading term (sec): 328.6792416572571
images per sec: 102.77047803895994
accuracy: {'construction_danger': 0.9, 'construction_safe': 1.0, 'park_danger': 1.0, 'park_safe': 0.9, 'road_danger': 0.8, 'road_safe': 0.6}


##### custom_efficientnetb1_saved_model_TFTRT_INT8

In [9]:
test('custom_efficientnetb1_saved_model_TFTRT_INT8', 'saved_model', class_names)

loading term (sec): 326.1881334781647
images per sec: 98.390604486216
accuracy: {'construction_danger': 0.9, 'construction_safe': 1.0, 'park_danger': 1.0, 'park_safe': 0.9, 'road_danger': 0.8, 'road_safe': 0.6}


## custom_efficientnetb2_saved_model

##### custom_efficientnetb2_saved_model

In [10]:
test('../saved_models/custom_efficientnetb2_saved_model', 'keras', class_names)

loading term (sec): 174.8209934234619
images per sec: 6.41629250338067
accuracy: {'construction_danger': 1.0, 'construction_safe': 0.8, 'park_danger': 1.0, 'park_safe': 1.0, 'road_danger': 1.0, 'road_safe': 0.8}


##### custom_efficientnetb2_saved_model_TFTRT_F32

In [11]:
test('custom_efficientnetb2_saved_model_TFTRT_F32', 'saved_model', class_names)

loading term (sec): 411.55945014953613
images per sec: 102.72076414399017
accuracy: {'construction_danger': 1.0, 'construction_safe': 0.8, 'park_danger': 1.0, 'park_safe': 1.0, 'road_danger': 1.0, 'road_safe': 0.8}


##### custom_efficientnetb2_saved_model_TFTRT_INT8

In [12]:
test('custom_efficientnetb2_saved_model_TFTRT_INT8', 'saved_model', class_names)

loading term (sec): 461.33589124679565
images per sec: 90.42291725831285
accuracy: {'construction_danger': 1.0, 'construction_safe': 0.8, 'park_danger': 1.0, 'park_safe': 1.0, 'road_danger': 1.0, 'road_safe': 0.8}
